In [1]:
import requests
import json
from datetime import datetime
import zipfile

In [41]:
url = 'https://donnees.roulez-eco.fr/opendata/instantane'
response = requests.get(url)


if response.status_code == 200:
    zip_path = './instantanes_files/zip/instantane_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.zip'
    with open(zip_path, 'wb') as f:
        f.write(response.content)
        print('File saved successfully.')

    output_folder = './instantanes_files/xml/'

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(output_folder)
        

    print('File unzipped successfully.')
else:
    print('Failed to download the file.')

File saved successfully.
File unzipped successfully.


In [43]:
!pip install xmltodict

In [45]:
#xml to json
import xmltodict
import json
import os

xml_folder = './instantanes_files/xml/'
json_folder = './instantanes_files/json/'

for filename in os.listdir(xml_folder):
    if filename.endswith(".xml"):
        xml_path = xml_folder + filename
        json_path = json_folder + filename.replace('.xml', '.json')
        with open(xml_path, 'r', encoding='ISO-8859-1') as xml_file:
            xml_content = xml_file.read()
            json_content = json.dumps(xmltodict.parse(xml_content), indent=4)
            with open(json_path, 'w') as json_file:
                json_file.write(json_content)
                print('File ' + filename + ' converted successfully.')
    else:
        continue



File PrixCarburants_instantane.xml converted successfully.
